In [2]:
import pandas as pd
import requests

FPS code information
https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt
25        MASSACHUSETTS

group information:
https://api.census.gov/data/2019/acs/acs5/profile/groups.html

In [151]:
def group_call(groupID):
    # getting meta data information for group
    meta = pd.read_csv('data/census_data/metadatagroup.csv', index_col='ID')
    meta = meta[meta['Group'] == groupID]
    meta.loc['NAME'] = ['','','municipality']
    estIDs = meta.index
    
    # Set base url
    url = 'https://api.census.gov/data/2019/acs/acs5/profile?'
    
    # Set params
    params = {
        'get': f'group({groupID})',
        'for': 'place:*',
        'in': 'state:25',
        'key': '91e51fe3baca2758aeb366be312da052c76caa13'
    }
    # Make a request and display the response code.
    res = requests.get(url,params)
    res
    # Create a dataframe from the request JSON object.
    df = pd.DataFrame(res.json())
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    
    # Only keep cols of interest
    df = df[estIDs]
    
    # Relabel IDs to Labels, format municipality names, and make muncipality the index
    df.rename(columns= dict(zip(df.columns, meta['label'])), inplace = True)
    df['municipality'] = df['municipality'].str.rstrip(', Massachusetts')
    df['municipality'] = df['municipality'].str.replace('Town city', '')
    df['municipality'] = df['municipality'].str.replace('CDP', '')
    df['municipality'] = df['municipality'].str.replace('city', '')
    df.set_index('municipality', inplace = True)
    
    return df

In [186]:
groups = {
    'education_char':'DP02',
    'economic_char':'DP03',
    'housing_char':'DP04',
    'demo_char':'DP05'
}

for characteristic in groups:
    group_call(groups[characteristic])
    exec(f'{characteristic} = group_call(groups[characteristic])')

In [187]:
education_char.columns

Index(['population_25_and_older', 'edu_high_school_and_higher_%',
       'edu_bachelors_and_higher_%'],
      dtype='object', name=0)

In [188]:
education_char.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, Weymouth  to Hanscom AFB 
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   population_25_and_older       245 non-null    object
 1   edu_high_school_and_higher_%  245 non-null    object
 2   edu_bachelors_and_higher_%    245 non-null    object
dtypes: object(3)
memory usage: 7.7+ KB


In [189]:
education_char['edu_bachelors_and_higher_%'] = pd.to_numeric(education_char['edu_bachelors_and_higher_%'])
education_char['edu_high_school_and_higher_%'] = pd.to_numeric(education_char['edu_high_school_and_higher_%'])
education_char['population_25_and_older'] = pd.to_numeric(education_char['population_25_and_older'])

In [190]:
education_char.describe()

,population_25_and_older,edu_high_school_and_higher_%,edu_bachelors_and_higher_%
count,245.000000,245.000000,245.000000
mean,13719.473469,93.365306,43.095510
std,34595.002572,5.501153,17.911639
min,34.000000,67.000000,9.600000
25%,1581.000000,90.800000,28.800000
50%,3398.000000,94.700000,42.400000
75%,16935.000000,97.400000,55.400000
max,473775.000000,100.000000,84.700000


In [191]:
economic_char.head()

,unemployment_rate,median_household_income,mean_household_income,families_below_poverty_level_%,people_below_poverty_level_%
municipality,,,,,
Weymouth,5.8,84942,100858,4.3,6.0
Winthrop,3.6,74069,102769,5.4,8.8
Belchertown,4.4,78578,96836,3.7,9.3
Needham,4.0,165547,223894,1.4,2.7
Quincy,5.1,77562,94360,8.0,11.3


In [194]:
for column in economic_char.columns:
    economic_char[column] = pd.to_numeric(economic_char[column])

In [195]:
economic_char.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, Weymouth  to Hanscom AFB 
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   unemployment_rate               245 non-null    float64
 1   median_household_income         245 non-null    int64  
 2   mean_household_income           245 non-null    int64  
 3   families_below_poverty_level_%  245 non-null    float64
 4   people_below_poverty_level_%    245 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 11.5+ KB


In [198]:
for column in demo_char.columns:
    demo_char[column] = pd.to_numeric(demo_char[column])

In [199]:
demo_char.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, Weymouth  to Hanscom AFB 
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   males-to-100females     245 non-null    float64
 1   median_age              245 non-null    float64
 2   demo_white_%            245 non-null    float64
 3   demo_black_aa_%         245 non-null    float64
 4   demo_american_indian_%  245 non-null    float64
 5   demo_asian_%            245 non-null    float64
 6   demo_native_islander%   245 non-null    float64
 7   demo_other_Race_%       245 non-null    float64
 8   demo_hispanic_latino_%  245 non-null    float64
dtypes: float64(9)
memory usage: 19.1+ KB


In [201]:
demo_char.sort_values(by='demo_white_%')

,males-to-100females,median_age,demo_white_%,demo_black_aa_%,demo_american_indian_%,demo_asian_%,demo_native_islander%,demo_other_Race_%,demo_hispanic_latino_%
municipality,,,,,,,,,
Randolph,92.8,40.7,34.7,47.5,0.7,13.2,0.0,7.1,8.8
Brockton,91.0,35.7,38.1,48.0,0.9,2.4,0.1,14.3,11.1
Lynn,101.5,34.5,54.0,18.2,1.1,8.3,0.3,25.3,42.8
Lawrence,95.8,32.3,55.2,7.6,0.5,1.9,0.3,37.6,80.6
Malden,91.8,34.6,55.7,20.0,0.6,23.8,0.2,3.1,8.5
...,...,...,...,...,...,...,...,...,...
Cheshire,100.4,51.8,100.0,0.0,1.5,0.0,0.0,0.0,0.0
Warren,78.9,42.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
Chatham,83.5,67.4,100.0,0.0,1.0,0.0,0.0,0.0,0.0


In [202]:
demo_char.describe()

,males-to-100females,median_age,demo_white_%,demo_black_aa_%,demo_american_indian_%,demo_asian_%,demo_native_islander%,demo_other_Race_%,demo_hispanic_latino_%
count,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000
mean,95.375102,45.424490,90.091020,4.671837,0.674286,4.402041,0.129796,2.768163,6.519184
std,25.984873,9.260783,10.691965,6.709394,1.073389,5.522936,0.397392,4.713189,9.812768
min,42.200000,20.400000,34.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,86.900000,40.200000,86.300000,0.800000,0.000000,0.500000,0.000000,0.000000,1.500000
50%,92.900000,43.800000,93.500000,2.800000,0.400000,2.400000,0.000000,1.200000,3.500000
75%,99.400000,49.900000,97.500000,5.900000,0.900000,6.000000,0.100000,3.300000,7.400000
max,443.900000,81.100000,100.000000,48.000000,9.200000,32.600000,4.700000,37.600000,80.600000


In [203]:
for column in housing_char.columns:
    housing_char[column] = pd.to_numeric(housing_char[column])

In [204]:
housing_char.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, Weymouth  to Hanscom AFB 
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Total housing units  245 non-null    int64  
 1   occupancy_%          245 non-null    float64
 2   1-unit_detached_%    245 non-null    float64
 3   1-unit_attached_%    245 non-null    float64
 4   2-units_%            245 non-null    float64
 5   3-4_units_%          245 non-null    float64
 6   5-9_units_%          245 non-null    float64
 7   10-19_units_%        245 non-null    float64
 8   20+_units_%          245 non-null    float64
 9   mobile_home_%        245 non-null    float64
 10  boat_RV_van_%        245 non-null    float64
 11  owner-occupied_%     245 non-null    float64
 12  renter-occupied_%    245 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 26.8+ KB


In [206]:
housing_char[housing_char['occupancy_%'] < 10]

,Total housing units,occupancy_%,1-unit_detached_%,1-unit_attached_%,2-units_%,3-4_units_%,5-9_units_%,10-19_units_%,20+_units_%,mobile_home_%,boat_RV_van_%,owner-occupied_%,renter-occupied_%
municipality,,,,,,,,,,,,,
Siasconset,1056,3.7,98.6,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0


In [208]:
# For saving files

# for char_df in groups:
#     exec(f"{char_df}.to_csv('data/census_data/{char_df}.csv', index=True)")